In [2]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

In [7]:
links = {}

with open('../language-pairs.json', 'r') as f:
    pairs = json.loads(f.read())
    
    for pair in pairs:
        links[pair] = pairs[pair]['wiki']

links

{'Hindi-Persian': 'https://en.m.wiktionary.org/wiki/Category:Hindi_terms_borrowed_from_Persian',
 'English-French': 'https://en.m.wiktionary.org/wiki/Category:English_terms_borrowed_from_French'}

In [15]:
base_link = "https://en.m.wiktionary.org"


In [16]:
def get_words_ipas(dest_lang, src_lang, link, page_num):
    borrowed_words = []

    try:
        # get list of words
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        res = soup.find_all("div", class_="mw-category-group")

        for i, r in enumerate(res):
            if i > 0:
                list_words = r.find_all('a')
                src_word = None

                for borrowed_word in list_words:

                    # for each loaned word get the wiki link and extract ipas for it.
                    borrowed_ipas = []
                    ipa_page = requests.get(base_link + borrowed_word['href'])
                    ipa_soup = BeautifulSoup(ipa_page.content, 'html.parser')
                    
                    # for ipa in ipa_soup.find_all("span", {'class': 'IPA'}):
                    #     txt = ipa.text
                    #     if txt.startswith('/') and txt.endswith('/'):
                    #         borrowed_ipas.append(ipa.text)
                            # print(word.text, ipa.text)  
                    
                    # get the src word and wiki link to extract ipas for the src word
                    src_word_html_final = None
                    for src_word_html in ipa_soup.find_all("span", class_="etyl"):
                        if src_word_html != None and src_word_html.find(lambda tag: tag.name == 'a' and src_lang.lower() in tag.text.lower()):
                            
                            src_word_html_final = src_word_html.find_next("i")
                            # src_word_link = src_word_html_final.find('a')
                            break
                    
                    # src_ipas = []
                    # if src_word_link != None:
                    #     src_word_link = src_word_link['href']
                    #     src_word_page = requests.get(base_link + src_word_link)
                    #     src_word_soup = BeautifulSoup(src_word_page.content, 'html.parser')
                    #     for ipa in src_word_soup.find_all("span", {'class': 'IPA'}):
                    #         txt = ipa.text
                    #         if txt.startswith('/')  and txt.endswith('/') :
                    #             src_ipas.append(ipa.text)

                    src_word = src_word_html_final.text if src_word_html_final != None else ''
                    
                    # save the result so far.
                    borrowed_words.append([borrowed_word.text, src_word])

        print(dest_lang, src_lang, page_num, "done", len(borrowed_words))
        
        # find whet her there is a next page containing more loan words if there is scrape that as well. 
        next_link = soup.find(
            lambda tag: tag.name == 'a' and 'next page' in tag.text.lower()
        )

        # return borrowed_words
        if next_link != None:
            more_words = get_words_ipas(dest_lang, src_lang, base_link + next_link['href'], page_num+1)
            borrowed_words = borrowed_words + more_words

    except Exception as e:
        # errors can be due to timeouts/connections refused due to rate limiting. can be fixed via proxies/vpns/timeouts
        print(pair, page_num, "got error", e, len(borrowed_words))
        return borrowed_words
        
    return borrowed_words

In [18]:
for pair in links:
    if os.path.exists("results/{}.csv".format(pair)):
        overwrite = input("{}.csv exists. Overwrite existing file? (y/n) ".format(pair))
    if overwrite == "y" or not os.path.exists("results/{}.csv".format(pair)):
        print()
        [dest, src] = pair.split('-')
        words = get_words_ipas(dest, src, links[pair], 1)
        df = pd.DataFrame(words, columns=['loan_word', 'original_word'])
        df.to_csv("results/{}.csv".format(pair), index=False)
        print(pair, "done\n")

Hindi-Persian.csv exists. Overwrite existing file? (y/n) n
English-French.csv exists. Overwrite existing file? (y/n) n
Finnish-Swedish.csv exists. Overwrite existing file? (y/n) n
Urdu-English.csv exists. Overwrite existing file? (y/n) n
French-English.csv exists. Overwrite existing file? (y/n) n
